
# PubMed API Script

This Jupyter notebook demonstrates how to use the PubMed API to fetch and process academic papers. 
The script includes functions for querying PubMed, fetching metadata, and identifying authors based on affiliations.

### Importing Required Libraries


In [9]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from typing import List, Dict, Tuple
import string


## Function Definitions

The following section contains functions for:
- Querying the PubMed API
- Parsing XML responses
- Processing and categorizing papers


### Construct Query URLs
The `mkquery` function helps in constructing a URL with query parameters for API requests.

In [10]:
def mkquery(base_url: str, params: Dict[str, str]) -> str:
    query = "&".join(f"{key}={value}" for key, value in params.items())
    return f"{base_url}?{query}"


### Fetch XML Data
The `getXmlFromURL` function sends a request to the PubMed API and parses the XML response.

In [11]:
def getXmlFromURL(base_url: str, params: Dict[str, str]) -> ET.Element:
    response = requests.get(mkquery(base_url, params))
    response.raise_for_status()
    return ET.fromstring(response.text)


### Fetch Paper IDs
The `fetch_paper_ids` function retrieves PubMed IDs for papers matching a given query.

In [12]:
def fetch_paper_ids(query: str, max_results: int = 100) -> Tuple[List[str], str, str]:
    params = {
        'db': 'pubmed',
        'term': query,
        'retmax': str(max_results),
        'usehistory': 'y'
    }
    root = getXmlFromURL(BASEURL_SRCH, params)
    ids = [id_node.text for id_node in root.findall('.//Id')]
    query_key = root.findtext('.//QueryKey')
    web_env = root.findtext('.//WebEnv')
    return ids, query_key, web_env


### Fetch Paper Details
The `fetch_paper_details` function retrieves detailed information about the papers using PubMed IDs.

In [13]:
def fetch_paper_details(query_key: str, web_env: str, batch_size: int = 10) -> List[Dict]:
    params = {
        'db': 'pubmed',
        'query_key': query_key,
        'WebEnv': web_env,
        'retmax': str(batch_size),
        'retmode': 'xml'
    }
    root = getXmlFromURL(BASEURL_FTCH, params)
    papers = []

    for article in root.iter('PubmedArticle'):
        paper = {
            'PubmedID': article.findtext('.//PMID'),
            'Title': article.findtext('.//ArticleTitle'),
            'PublicationDate': article.findtext('.//PubDate/Year'),
            'Authors': [],
            'Affiliations': []
        }
        for author in article.findall('.//Author'):
            name = f"{author.findtext('ForeName', '')} {author.findtext('LastName', '')}".strip()
            affiliation = author.findtext('.//Affiliation')
            if name and affiliation:
                paper['Authors'].append(name)
                paper['Affiliations'].append(affiliation)
        papers.append(paper)
    return papers


### Check Academic Affiliation
The `check_academic` function determines if an affiliation is academic by matching keywords.

In [14]:
def check_academic(affiliation: str) -> bool:
    academic_keywords = ["school", "university", "college", "institute", "research", "lab"]
    affiliation = affiliation.translate(str.maketrans('', '', string.punctuation))
    affiliation = affiliation.lower().split()
    academic_keywords = set(academic_keywords)
    affiliation = set(affiliation)
    return len(academic_keywords.intersection(affiliation)) > 0


### Process Papers
The `process_papers` function processes metadata and categorizes authors as academic or non-academic.

In [15]:
def process_papers(papers: List[Dict]) -> pd.DataFrame:
    rows = []
    for paper in papers:
        non_academic_authors = []
        affiliations = []
        for author, affiliation in zip(paper['Authors'], paper['Affiliations']):
            is_academic = check_academic(affiliation)
            if not is_academic:
                non_academic_authors.append(author)
                affiliations.append(affiliation)
        rows.append({
            'PubmedID': paper['PubmedID'],
            'Title': paper['Title'],
            'Publication Date': paper['PublicationDate'],
            'Non-academic Author(s)': "; ".join(non_academic_authors),
            'Company Affiliation(s)': "; ".join(affiliations),
        })
    return pd.DataFrame(rows)



## Example Usage

The example demonstrates querying PubMed for papers and processing results.


In [18]:

# Constants for PubMed API
BASEURL_SRCH = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
BASEURL_FTCH = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'

# Example query
query = "machine learning"

# Fetch PubMed IDs
ids, query_key, web_env = fetch_paper_ids(query)
print(f"Fetched {len(ids)} IDs")

# Fetch details
papers = fetch_paper_details(query_key, web_env)
print(f"Fetched details for {len(papers)} papers")

# Process papers
df = process_papers(papers)

df


Fetched 100 IDs
Fetched details for 10 papers


,PubmedID,Title,Publication Date,Non-academic Author(s),Company Affiliation(s)
0,39791229,Role of data-driven regional growth model in s...,2025,,
1,39791216,Predicting cognitive decline from neuropsychia...,2025,Janina Krell-Roesch; David S Knopman; Cliff R ...,"Department of Quantitative Health Sciences, Ma..."
2,39791175,Identification of Programmed Cell Death-relate...,2025,,
3,39791105,The impact of war on people with type 2 diabet...,2025,Oksana Sulaieva; Volodymyr Pankiv; Oleksandr T...,"Medical Laboratory CSD, Kyiv 02000, Ukraine.; ..."
4,39791061,Differentiating Cystic Lesions in the Sellar R...,2024,Kaivan Patel,"Department of Internal Medicine, Broward Healt..."
5,39791019,Retraction: Application of Metabolomics and Ma...,2025,,
6,39790908,Perceptions of Machine Learning among Therapis...,2024,,
7,39790862,Diagnostic performance of generative pretraine...,2025,,
8,39790563,Identification of fibrosis-related genes and b...,2024,,
9,39790466,Expanding the recognition of monosaccharides a...,2025,,
